# 🗺️ Planners (with Handlebars planner)

### Initialize the kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.7.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.7.1-preview"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, 1.7.1-preview"

#!import Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var kernel = Settings.InitializeKernel();

Installed Packages Microsoft.SemanticKernel, 1.7.1 Microsoft.SemanticKernel.Abstractions, 1.7.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.7.1-preview Microsoft.SemanticKernel.Planners.OpenAI, 1.7.1-preview Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha Microsoft.SemanticKernel.Plugins.Web, 1.7.1-alpha

### 🔌 Let's add a native plugin for use by the planner

In [2]:
#!import Helpers/PrintHelper.cs
#!import Plugins/MotorPlugin.cs

kernel.Plugins.AddFromType<MotorPlugin>();

PrintHelper.PrintAllPluginFunctions(kernel);

****** Registered 🔌 Plugins and 📦 Functions ******
🔌 MotorPlugin
   📦 /Backward: Moves the car backward.
🔌 MotorPlugin
   📦 /Forward: Moves the car forward.
🔌 MotorPlugin
   📦 /Stop: Stops the car.
🔌 MotorPlugin
   📦 /TurnLeft: Turns the car anticlockwise.
🔌 MotorPlugin
   📦 /TurnRight: Turns the car clockwise.


### 📋 Prepare kernel arguments

In [3]:
var goal = "You have a tree in front of the car. Avoid it.";
var kernelArguments = new KernelArguments()
{
    ["input"] = goal,
    ["commands"] = "go forward, go backward, turn left, turn right, and stop"
};

### Refine the goal

In [4]:
var commandsPlugin = kernel.CreatePluginFromPromptDirectory(System.IO.Path.Combine(System.IO.Directory.GetCurrentDirectory(), "Plugins", "CommandsPlugin"), "CommandsPlugin");
kernel.Plugins.Add(commandsPlugin);
PrintHelper.PrintAllPluginFunctions(kernel);

****** Registered 🔌 Plugins and 📦 Functions ******
🔌 MotorPlugin
   📦 /Backward: Moves the car backward.
🔌 MotorPlugin
   📦 /Forward: Moves the car forward.
🔌 MotorPlugin
   📦 /Stop: Stops the car.
🔌 MotorPlugin
   📦 /TurnLeft: Turns the car anticlockwise.
🔌 MotorPlugin
   📦 /TurnRight: Turns the car clockwise.
🔌 CommandsPlugin
   📦 /ExtractBasicCommands: Break down complex actions into basic motor commands.
      📥 Params:
       • input: Complex action to be break down into basic motor commands. (default: '')
       • commands: The basic motor commands to choose from. (default: '')


In [5]:
var semanticFunction = commandsPlugin["ExtractBasicCommands"];
var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);
var refinedGoal = response.GetValue<string>();

kernelArguments["input"] = refinedGoal;

display(refinedGoal)

To avoid the tree in front of the car and return to a stopped position, the following sequence of basic motor commands can be used:

- Go forward (to start moving away from the initial stopped position)
- Turn right (to start veering away from the tree)
- Go forward (to pass beside the tree)
- Turn left (to realign with the original direction past the tree)
- Go forward (to create some distance from the tree)
- Turn left (to make a U-turn)
- Go forward (to head back towards the initial position)
- Turn left (to face the original direction)
- Go forward (to approach the initial position)
- Stop (to reach the final state of being stopped)

### 🗺️ Generate Handlebars Plan

In [6]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var handlebarsPlannerOptions = new HandlebarsPlannerOptions 
{ 
    AllowLoops = true 
};
handlebarsPlannerOptions.ExcludedPlugins.Add("CommandsPlugin");
var planner = new HandlebarsPlanner(handlebarsPlannerOptions);
var plan = await planner.CreatePlanAsync(kernel, refinedGoal);

//Console.WriteLine($"The proposed plan in Handlebars format:\n{plan}");

### 🗺️🚲 Let's run the plan!

(Note that the final output is json but the planner strips the extra JSON syntax)

In [7]:
#pragma warning disable SKEXP0060

var planResult = await plan.InvokeAsync(kernel, kernelArguments);
Console.WriteLine(planResult.Trim());

🡲
🡶
🡲
🡵
🡲
🡵
🡲
🡵
🡲
·
The car has avoided the tree and returned to a stopped position.
